# Feature engineering
## Milestone 2

### Dominik Pawlak, Przemysław Olender

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import category_encoders as ce
import warnings
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from statistics import stdev
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [ ]:
grades_df = pd.read_csv('school_grades_dataset.csv')

print(grades_df.shape)
grades_df.head()

## Regresja liniowa dla G1 i G2

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (12, 5))

sns.scatterplot(data = grades_df, x = 'G1', y = 'G3', ax = ax1)
sns.scatterplot(data = grades_df, x = 'G2', y = 'G3', ax = ax2)

plt.show()

# liniowa zalezcnosc miedzy G1, G2, i G3

Sprawdźmy jak sprawdzi się prsoty model regresji liniowej dla G1, G2 oraz G1 i G2 jednocześnie.

In [ ]:
X1 = grades_df[['G1']]
Y = grades_df['G3']

model1 = LinearRegression()
X1_train, X1_test, Y1_train, Y1_test = train_test_split(X1, Y, test_size = 0.3, random_state = 1)
model1.fit(X1_train, Y1_train)
Y1_test_predicted = model1.predict(X1_test)

In [ ]:
plt.scatter(X1_test, Y1_test, label = 'testing data')
plt.plot(X1_test, Y1_test_predicted, label = 'prediction', linewidth = 3, color = 'red')
plt.xlabel('G1')
plt.ylabel('G3')
plt.legend(loc = 'upper left')
plt.show()

In [ ]:
print(f'RMSE: {mean_squared_error(Y1_test, Y1_test_predicted, squared = False)}')
print(f'R-squared: {model1.score(X1_test, Y1_test)}')

In [ ]:
X2 = grades_df[['G2']]
model2 = LinearRegression()
X2_train, X2_test, Y2_train, Y2_test = train_test_split(X2, Y, test_size = 0.3, random_state = 1)
model2.fit(X2_train, Y2_train)
Y2_test_predicted = model2.predict(X2_test)

In [ ]:
plt.scatter(X2_test, Y2_test, label = 'testing data')
plt.plot(X2_test, Y2_test_predicted, label = 'prediction', linewidth = 3, color = 'red')
plt.xlabel('G2')
plt.ylabel('G3')
plt.legend(loc = 'upper left')
plt.show()

In [ ]:
print(f'RMSE: {mean_squared_error(Y2_test, Y2_test_predicted, squared = False)}')
print(f'R-squared: {model2.score(X2_test, Y2_test)}')

In [ ]:
X3 = grades_df[['G1', 'G2']]
model3 = LinearRegression()
X3_train, X3_test, Y3_train, Y3_test = train_test_split(X3, Y, test_size = 0.3, random_state = 1)
model3.fit(X3_train, Y3_train)
Y3_test_predicted = model3.predict(X3_test)

In [ ]:
print(f'RMSE: {mean_squared_error(Y3_test, Y3_test_predicted, squared = False)}')
print(f'R-squared: {model3.score(X3_test, Y3_test)}')

Najlepeij wypadła regresja oparta na G1 i G2, model oparty tylko na G2 jest minimalnie gorszy, najsłabeij prezentuje się model oparty na G1.

## Przewidywanie G3 za pomocą innych zmiennych oraz bez G1 i G2

In [ ]:
grades_df[grades_df['G3'] == 0]

Cieżko jest nie zdobyć żadnego punktu na egazminie, zakładamy, że te osoby nie podeszły i nie można przewidzieć ilości punktów, usuwamy rekordy  z ramki.

In [ ]:
grades_df = grades_df[grades_df['G3'] != 0]

grades_df.shape

 W naszej ramce mamy też dużo zmienncyh kategorycznych, zakodujmy je, żeby usprawnić działanie modeli.

In [ ]:
cat_cols = ['school', 'sex', 'address', 'famsize', 'Mjob', 'Fjob', 'reason', 'guardian', 'Pstatus', 'sex', 'school']
bin_cols = ['famsup', 'activities', 'nursery', 'internet', 'romantic', 'higher', 'paid', 'schoolsup']

grades_df_new = grades_df.drop(columns = cat_cols)
grades_df_new = grades_df.drop(columns = bin_cols)

for i in cat_cols:
    means = grades_df.groupby(i)['G3'].mean()
    grades_df_new[i] = grades_df[i].map(means)
    
for i in bin_cols:
    encoder = ce.OrdinalEncoder(mapping = [{'col': i, 'mapping': {'yes': 1, 'no': 0}},])
    grades_df_new[i] = encoder.fit_transform(grades_df)[i]

Pogrupujmy rezultat G3 w przedział, aby nie trzeba było przewidywać dokłądniej liczby - zadanie będzie prostsze.

In [ ]:
names = ['failed', 'weak', 'ok', 'good', 'excellent']

grades_df_new['result'] = pd.cut(grades_df['G3'], bins=[-1, 9, 11, 13, 15, 21], labels = names)

sns.countplot(grades_df_new['result'], order = names)

In [ ]:
# funkcja liczby accuracy dla 4 modeli i za pomocą xgboost wybiera najważniejsze cechy

def modelScores(X, X_train, X_test, y_train, y_test):
    
    importance = pd.DataFrame()
    importance['col'] = X.columns 
    importance['xgb'] = 0

    model_xgb = xgb.XGBClassifier(
                    max_depth=4
                    ,learning_rate=0.2
                    ,reg_lambda=1
                    ,n_estimators=150
                    ,subsample = 0.9
                    ,colsample_bytree = 0.9)
    model_xgb.fit(X_train, y_train)

    #print(model_xgb.feature_importances_)
    #print(sum(model_xgb.feature_importances_))
    #print(len(model_xgb.feature_importances_))
    #print(grades_df_new.shape)

    importance['xgb'] = importance['xgb'] + model_xgb.feature_importances_ / 100
    importance = importance.sort_values(axis=0, ascending=False, by='xgb')
    #print('Top 10 features:')
    #print(importance.iloc[:, 1:])
    print(importance.reset_index().drop('index', axis = 1))
    
    print(f'xgboost accuarcy: {model_xgb.score(X_test,y_test)}')
    
    modelLR = LogisticRegression(random_state=1, max_iter=100000)
    modelLR.fit(X_train, y_train)
    y_hat = modelLR.predict(X_test)
    print(f'Logistic Regression accuracy: {accuracy_score(y_test, y_hat)}')
    
    model_rf = RandomForestClassifier(n_estimators=188,
                                  max_depth=5,
                                  min_samples_split = 7, 
                                  max_features = len(X_train.columns),
                                  random_state=0,
                                  n_jobs = 15)
    model_rf.fit(X_train, y_train)
    print(f'RandomForestClassifier accuracy: {model_rf.score(X_test,y_test)}')
    
    model1 = DecisionTreeClassifier(random_state=1)
    clf = BaggingClassifier(base_estimator=model_rf,
                        n_estimators=100, random_state=0)
    clf.fit(X_train, y_train)
    print(f'DecisionTreeClassifier accuracy: {clf.score(X_test,y_test)}')
    
    return importance['col'].head(10)

In [ ]:
X = grades_df_new.drop(['G3', 'result'], axis = 1)
Y = grades_df_new['result']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)
#X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.3, random_state=42)

#print(X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape)

In [ ]:
cols = modelScores(X, X_train, X_test, y_train, y_test)

In [ ]:
X = grades_df_new[cols]
Y = grades_df_new['result']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)
#X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.3, random_state=42)

#print(X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape)
modelScores(X, X_train, X_test, y_train, y_test)

In [ ]:
X = grades_df_new.drop(['G3', 'result'], axis = 1)
Y = grades_df_new['result']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)
#X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.3, random_state=42)

#print(X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape)

Wyniki gorsze niż dla prostej regresji liniowej.

Stworzylismy prosty model opierający sięn a G1 i G2, spróbujmy teraz bez tych atrybutów.

In [ ]:
grades_df_new = grades_df_new.drop(columns = ['G1', 'G2'])

Przewidywanie przedziału oceny za pomocą wszystkich kolumn:

In [ ]:
X = grades_df_new.drop(['G3', 'result'], axis = 1)
Y = grades_df_new['result']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)
#X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.3, random_state=42)

#print(X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape)

In [ ]:
cols = modelScores(X, X_train, X_test, y_train, y_test)

Przewidywanie przedziału oceny za pomocą najważnijeszych kolumn:

In [ ]:
X = grades_df_new[cols]
Y = grades_df_new['result']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)
#X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.3, random_state=42)

#print(X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape)
modelScores(X, X_train, X_test, y_train, y_test)

Przewidywanie dokładnej oceny za pomocą wszystkich kolumn:

In [ ]:
X = grades_df_new.drop(['G3', 'result'], axis = 1)
Y = grades_df_new['G3']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)
#X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.3, random_state=42)

#print(X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape)

cols = modelScores(X, X_train, X_test, y_train, y_test)

Przewidywanie dokładnej oceny za pomocą najważnieszych kolumn:

In [ ]:
X = grades_df_new[cols]
Y = grades_df_new['G3']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)
#X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.3, random_state=42)

#print(X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape)
modelScores(X, X_train, X_test, y_train, y_test)